# ComputerVision - Image Recognition / Classification

### Image Classification Using CNNs

## Loading the data

From the Google website: https://quickdraw.withgoogle.com/

In [ ]:
import random 
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import save
from numpy import load
from PIL import Image
import png
import os
from vector_to_raster import vector_to_raster
import json
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from keras import models
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import pickle

In [ ]:
def load_data(animal, side, n_images):
    """This function transforms a json file (with raw data of images) 
    into lists of 1,000 np.arrays with shape 256*256 (tensors of pixels),
    each representing a drawing of an animal 
    Input = 1 json file
    Output = 1,000 np.arrays"""
    animal_data = [json.loads(line) for line in open(f'raw_data/full_simplified_{animal}.ndjson', 'r')]
    animal_data = random.choices(animal_data, k=n_images)
    vector_images = [element['drawing'] for element in animal_data]
    data = vector_to_raster(vector_images, side=side, line_diameter=16, padding=16, bg_color=(1,1,1), fg_color=(0,0,0))
    return data

In [ ]:
def save_png(type_animal, data):
    """This function transforms np.arrays of pixels with shape 256*256 
    into .png (real images of resolution 256*256) and saves them in a subdirectory.
    Input = 1, 000 np. arrays 
    Output = 1,000 drawings of one type of animal (images.png)"""
    path = f'images/{type_animal}'
    if type_animal not in os.listdir('images/'):
        os.mkdir(path)
    for i in range(len(data)):
        img = data[i]
        png.from_array(np.array(img), 'L').save(f'{path}/img_{i}.png')

In [ ]:
def plot_samples(X, y, rows=1, cols=5, title=''):
    X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
    input_data = np.c_[X, y]
    fig, ax = plt.subplots(figsize=(cols,rows))
    ax.axis('off')
    plt.title(title)

    for i in (range(0, min(len(input_data),(rows*cols)))):      
        a = fig.add_subplot(rows,cols,i+1)
        imgplot = plt.imshow(input_data[i,:65536].reshape((256,256)), cmap='gray_r', interpolation='nearest')
        plt.xticks([])
        plt.yticks([])

In [ ]:
def resizing_X(X_orig, small_side):
    """This function aims at resizing the original features X 
    (256*256) into smaller dimensions."""
    X = []
    for i in range(X_orig.shape[0]):
        X.append([])
        for j in range(X_orig.shape[1]):
            X[-1].append(cv2.resize(X_orig[i, j,], dsize=(small_side, small_side), interpolation=cv2.INTER_CUBIC))
    return np.array(X)

In [ ]:
# dict with 2 classes of animals
animal_dict_2 = { 
    'cow': 0,
    'panda': 1, 
}

# dict with 4 classes of animals
animal_dict_4 = {
    'frog': 0,
    'rabbit': 1,  
    'elephant': 2,
    'duck': 3,
}

# dict of 8 classes of animals
animal_dict_8 = {
    'camel': 0,
    'rabbit': 1,  
    'frog': 2,
    'duck': 3,
    'elephant': 4, 
    'cow': 5, 
    'shark': 6, 
    'panda': 7
}

To save the data as png images (optional), uncomment the command line #save_png below:

In [ ]:
n_images = 10000
side = 256

X = []
y = []

In [ ]:
animal_dict = animal_dict_8
nb_classes = len(animal_dict)

In [ ]:
for animal in animal_dict:
    # Load the data into numpy arrays
    print(f'Animal {animal} | Loading data into numpy array')
    data = load_data(animal, side, n_images)
    # Save numpy arrays as npy files
    print(f'Animal {animal} | Saving numpy array as npy')
    save(f'npy/animals/{animal}.npy', animal)
    # Save the data as images (.png)
    #print(f'Animal {animal} | Saving numpy array as png')
    #save_png(animal, data)
    # Build features and labels
    print(f'Animal {animal} | Building features and labels')
    X_animal = np.array(data)
    X.append(X_animal)
    X_orig = X
    # Creating labels y_animal for each type of animal and appending it to a y tensor
    y_animal = np.zeros(n_images,) + animal_dict[animal]
    y.append(y_animal)
    print(f'Animal {animal} | Plotting samples')
    title =(f'Sample of {animal} drawings')
    plot_samples(X_animal, y_animal, rows=1, cols=5, title=title)
X_orig = np.array(X_orig)
X = np.array(X)
y = np.array(y, dtype='uint8')

Uncomment the code below to save the features and labels.

In [ ]:
# Save the features and the labels
np.save('npy/features_labels/X_orig.npy', X_orig)
np.save('npy/features_labels/y.npy', y)

In [ ]:
# Re-load the data faster, when the code above has been run once.
#X_orig = np.load('npy/features_labels/X_orig.npy')
#y = np.load('npy/features_labels/y.npy')

## Build a Regular Neural Network (fully connected)

In [ ]:
# Reshape the data
side = 256
small_side = 56
X = resizing_X(X_orig, small_side)

X_orig = X_orig.reshape(X_orig.shape[0] *X_orig.shape[1], side, side)
X = X.reshape(X.shape[0] * X.shape[1], small_side, small_side)
y = y.reshape(y.shape[0] * y.shape[1])

In [ ]:
# Split the dataset into train and test sets (manually)
test_size = 0.33
train_images, test_images, train_images_orig, test_images_orig, train_labels, test_labels = train_test_split(X, X_orig, y, test_size = test_size, random_state = 69, shuffle = True)

# Prepare the data
train_images = train_images.reshape(train_images.shape[0], small_side*small_side)
train_images = train_images.astype('float32')/255
test_images = test_images.reshape(test_images.shape[0], small_side*small_side)
test_images = test_images.astype('float32')/255

# Build the network architecture (fully connected)
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(small_side*small_side,)))
network.add(layers.Dense(nb_classes, activation='softmax'))

opt = RMSprop(learning_rate=0.0001)

# Compile the model
network.compile(optimizer=opt, 
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
# Fit the model
history = network.fit(train_images, 
                      train_labels, 
                      epochs=30,
                      batch_size=128, 
                      validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the model
scores = network.evaluate(test_images, test_labels, verbose=0)
scores = round(scores[1]*100, 2)
print('Final network accuracy: ', scores, "%")

In [ ]:
# Predict classes for new data
for i in 20+np.array(range(10)):
    pred = list(animal_dict)[network.predict_classes(test_images)[i]]
    plt.imshow(test_images_orig[i].reshape((side, side)), cmap=plt.cm.binary)
    plt.title(f'Prediction: {pred}')
    plt.show()

### Learning curves of the model

In [ ]:
# Accuracy of the model
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Loss of the model
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

### K-fold Cross-Validation 

In [ ]:
X = X.reshape(X.shape[0], small_side*small_side)
X = X.astype('float32')/255
y = y.astype('uint8')

In [ ]:
kfold = KFold(n_splits=5, random_state=None, shuffle=True)

cvscores = []
split_nb = 0
for train, test in kfold.split(X, y):
    split_nb += 1
    print('Split n°', split_nb)
    # Create the model
    k_network = models.Sequential()
    k_network.add(layers.Dense(512, activation='relu', input_shape=(small_side*small_side,)))
    k_network.add(layers.Dense(nb_classes, activation='softmax'))
    # Compile the model
    k_network.compile(optimizer='rmsprop', 
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
    # Fit the model
    k_network.fit(X[train], y[train], epochs=30, batch_size=128, verbose=0)
    # evaluate the model
    scores = k_network.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (k_network.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("\n%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

## Build the CNN model

In [ ]:
# Re-load the data
X_orig = np.load('npy/features_labels/X_orig.npy')
y = np.load('npy/features_labels/y.npy')

In [ ]:
# Reshape the data
side = 256
small_side = 28
X = resizing_X(X_orig, small_side)

X_orig = X_orig.reshape(X_orig.shape[0] *X_orig.shape[1], side, side)
X = X.reshape(X.shape[0] * X.shape[1], small_side, small_side)
y = y.reshape(y.shape[0] * y.shape[1])

In [ ]:
# Split the dataset into train and test sets
test_size = 0.33
train_images, test_images, train_images_orig, test_images_orig, train_labels, test_labels = train_test_split(X, X_orig, y, test_size = test_size, random_state = 0, shuffle = True)

# Pre-process the data
train_images = train_images.reshape(train_images.shape[0], small_side, small_side, 1)
train_images = train_images.astype('float32')/255

test_images = test_images.reshape(test_images.shape[0], small_side, small_side, 1)
test_images = test_images.astype('float32')/255

In [ ]:
def convnet_model(nb_classes, small_side):
    """This function is the architecture of the CNN model."""
    convnet = models.Sequential()
    convnet.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(small_side, small_side, 1)))
    convnet.add(layers.MaxPooling2D((2, 2)))
    convnet.add(layers.Conv2D(128, (3, 3), activation='relu'))
    convnet.add(layers.MaxPooling2D((2, 2)))
    convnet.add(layers.Dropout(0.4))
    convnet.add(layers.Flatten())
    convnet.add(layers.Dense(128, activation='relu'))
    convnet.add(layers.Dense(50, activation='relu'))
    convnet.add(layers.Dense(nb_classes, activation='softmax'))

    optimizer = Adam(lr= 0.001)
    convnet.compile(optimizer='adam', 
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    return convnet

In [ ]:
# CNN with 2 classes
convnet_2 = convnet_model(nb_classes, small_side)
history_2 = convnet_2.fit(train_images, train_labels, epochs=10, batch_size=128, validation_data=(test_images, test_labels))

In [ ]:
# CNN with 4 classes of animals
convnet_1 = convnet_model(nb_classes, small_side)
history_1 = convnet_1.fit(train_images, train_labels, epochs=15, batch_size=128, validation_data=(test_images, test_labels))

In [ ]:
# CNN with 8 classes of animals
convnet_3 = convnet_model(nb_classes, small_side)
history_3 = convnet_3.fit(train_images, train_labels, epochs=20, batch_size=128, validation_data=(test_images, test_labels))

### Evaluate the model

In [ ]:
# Scores of the model with 2 classes
scores_2 = convnet_2.evaluate(test_images, test_labels, verbose=0)
scores_2 = round(scores_2[1]*100, 2)
print('Final CNN accuracy: ', scores_2, "%")

In [ ]:
# Scores of the model with 4 classes
scores_1 = convnet_1.evaluate(test_images, test_labels, verbose=0)
scores_1 = round(scores_1[1]*100, 2)
print('Final CNN accuracy: ', scores_1, "%")

In [ ]:
# Scores of the model with 8 classes
scores_3 = convnet_3.evaluate(test_images, test_labels, verbose=0)
scores_3 = round(scores_3[1]*100, 2)
print('Final CNN accuracy: ', scores_3, "%")

### Save convnet and convnet history to disk

In [ ]:
def save_model(model, i, history):
    """This function aims at saving a model architecture, as well as its weights and history."""
    model.save_weights(f'history/convnet_weights_{i}.h5')
    model.save(f'history/convnet_archi_{i}.model')
    with open(f'history/convnet_{i}.history', f'wb') as f:
        pickle.dump(history.history, f)
    print("Model has been saved")

In [ ]:
# With 8 animals
save_model(convnet_3, nb_classes, history_3)

### Plot learning curves

In [ ]:
/convnet_3.summary()

In [ ]:
# With 8 classes
plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Test set'], loc='lower right')
plt.show()

In [ ]:
# With 8 classes
plt.plot(history_3.history['loss']) 
plt.plot(history_3.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Training set', 'Test set'], loc='lower right')
plt.show()

### Confusion matrix

In [ ]:
y_true, y_pred = test_labels, np.argmax(convnet_3.predict(test_images), axis=-1)
cm = confusion_matrix(y_true, y_pred)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))     
ax = plt.subplot()
sns.heatmap(cm, annot=True, cmap="Greens", fmt='g')
ax.set_xlabel('\nPredicted labels', fontsize=14)
ax.set_ylabel('\nTrue labels', fontsize=14)
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(list(animal_dict))
ax.yaxis.set_ticklabels(list(animal_dict))
plt.show()

### Prediction dataframe

In [ ]:
y_df = pd.DataFrame(zip(y_true, y_pred))
y_df.columns = ['true', 'pred']
y_df['true_label'] = y_df['true'].apply(lambda x: list(animal_dict)[x])
y_df['pred_label'] = y_df['pred'].apply(lambda x: list(animal_dict)[x])
y_df.head(50)

### Visualizations

In [ ]:
def pred_visualization(nb_animals, animal_dict, model):
    for i in np.array(range(nb_animals)):
        pred = list(animal_dict)[model.predict_classes(test_images)[i]]
        pred_proba = max(model.predict(test_images)[i])*100
        pred_proba = round(pred_proba, 2)
        print(f'The drawing is identified as "{pred}" with probabiliy ' + str(pred_proba) + '%.')
        plt.imshow(test_images_orig[i].reshape((side, side)), cmap=plt.cm.binary)
        plt.title(f'Prediction: {pred}')
        plt.show()

In [ ]:
pred_visualization(10, animal_dict, convnet_3)

### Hyperparameter Tuning

Tuning learn rate and number of neurons in the hidden Layers using GridSearchCV.

In [ ]:
def create_model(neurons, learn_rate=0.01):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(small_side, small_side, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(nb_classes, activation='softmax'))

    optimizer = Adam(learn_rate)
    model.compile(optimizer=optimizer, 
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    return model

In [ ]:
# Use a classifier
model = KerasClassifier(build_fn=create_model)
# Define the grid_search parameters
learn_rate = [0.001, 0.01, 0.1]
neurons = [16, 32, 64]
param_grid = dict(learn_rate=learn_rate, neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_images, train_labels, epochs=15)
# Summarize results
print("Best: %s using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(test_images, test_labels))

### Loading and Testing new Data

To test a new image, upload a png file in the command np.array(Image.open(file.png)). You can draw an animal using this website: https://sketchpad.app/.

In [ ]:
drawing_orig = np.array(Image.open('test_new_drawings/drawing_3.png').convert('L'))
drawing = cv2.resize(drawing_orig, dsize=(small_side, small_side), interpolation=cv2.INTER_CUBIC)

drawings = np.array([drawing])
drawings = drawings.reshape(drawings.shape[0], small_side, small_side, 1)
drawings = drawings.astype('float32')/255
drawing_preds = list(convnet_3.predict_classes(drawings))

pred = list(animal_dict)[drawing_preds[0]]
plt.imshow(drawing_orig, cmap=plt.cm.binary)
plt.title('Prediction is: '+ pred)
plt.show()